In [32]:
import numpy as np
import pandas as pd

### EDA Conclusion
1. *The dataset is a clean time-series dataset with daily sales records from 2013 to 2017, containing 9,13,000 rows and 4 columns.*
2. *The dataset includes the features date, store (10 unique stores), item (50 unique items), and sales, with no missing or duplicate values.*
3. *Sales exhibit a clear upward trend over the years, indicating increasing demand over time.*
4. *Strong yearly seasonality is observed, with repeating demand patterns across years, confirming that sales are non-stationary.*
5. *Most daily sales values fall within the range of 0–100 units, representing regular demand levels across stores and items.*
6. *A small number of days experience very high sales (approximately 100–180 units), creating a long right tail in the distribution and              resulting in a right-skewed sales distribution.*
7. *All stores contain outliers, indicating that these high-sales values represent genuine demand spikes rather than data errors; therefore,         outliers should be retained, and store-specific modeling is likely to improve forecasting performance.*


## Feature Engineering

In [33]:
df = pd.read_csv("../data/processed/cleaned_data.csv",parse_dates=['date'])

In [34]:
df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-01,7,12,26
2,2013-01-01,7,46,27
3,2013-01-01,8,12,54
4,2013-01-01,9,12,35


## 1. Date Based Features

In [35]:
df["date"].dtype

dtype('<M8[ns]')

In [36]:
df = df.sort_values(["store","item","date"])

*Lag features depend on correct order,
Prevents data leakage (required for time series)*

In [37]:
df

,date,store,item,sales
0,2013-01-01,1,1,13
699,2013-01-02,1,1,11
1062,2013-01-03,1,1,14
1861,2013-01-04,1,1,13
2291,2013-01-05,1,1,10
...,...,...,...,...
910872,2017-12-27,10,50,63
911260,2017-12-28,10,50,59
911707,2017-12-29,10,50,74
912007,2017-12-30,10,50,62


#### *Creating derived input features*

In [38]:
df["year"] = df["date"].dt.year

In [39]:
df["month"] = df["date"].dt.month 

In [40]:
df["week"] = df["date"].dt.isocalendar().week

In [41]:
# date - 2017-12-27 - 27
df["day"] = df["date"].dt.day

In [42]:
# Monday(1) , Tuesday(2)
df["dayofweek"] = df["date"].dt.dayofweek

In [43]:
df["is_weekend"] = df["dayofweek"].isin([5,6]).astype(int) 

In [44]:
df

,date,store,item,sales,year,month,week,day,dayofweek,is_weekend
0,2013-01-01,1,1,13,2013,1,1,1,1,0
699,2013-01-02,1,1,11,2013,1,1,2,2,0
1062,2013-01-03,1,1,14,2013,1,1,3,3,0
1861,2013-01-04,1,1,13,2013,1,1,4,4,0
2291,2013-01-05,1,1,10,2013,1,1,5,5,1
...,...,...,...,...,...,...,...,...,...,...
910872,2017-12-27,10,50,63,2017,12,52,27,2,0
911260,2017-12-28,10,50,59,2017,12,52,28,3,0
911707,2017-12-29,10,50,74,2017,12,52,29,4,0
912007,2017-12-30,10,50,62,2017,12,52,30,5,1


### 2. Creating Lag and Rolling Features

In [45]:
# Lag Features
lags = [1, 7, 14, 28]

for lag in lags:
    df[f"sales_lag_{lag}"] = (
        df.groupby(['store', 'item'])['sales']
          .shift(lag)
    )

In [46]:
df.head(10)

,date,store,item,sales,year,month,week,day,dayofweek,is_weekend,sales_lag_1,sales_lag_7,sales_lag_14,sales_lag_28
0,2013-01-01,1,1,13,2013,1,1,1,1,0,NaN,NaN,NaN,NaN
699,2013-01-02,1,1,11,2013,1,1,2,2,0,13.0,NaN,NaN,NaN
1062,2013-01-03,1,1,14,2013,1,1,3,3,0,11.0,NaN,NaN,NaN
1861,2013-01-04,1,1,13,2013,1,1,4,4,0,14.0,NaN,NaN,NaN
2291,2013-01-05,1,1,10,2013,1,1,5,5,1,13.0,NaN,NaN,NaN
2659,2013-01-06,1,1,12,2013,1,1,6,6,1,10.0,NaN,NaN,NaN
3238,2013-01-07,1,1,10,2013,1,2,7,0,0,12.0,NaN,NaN,NaN
3587,2013-01-08,1,1,9,2013,1,2,8,1,0,10.0,13.0,NaN,NaN
4256,2013-01-09,1,1,12,2013,1,2,9,2,0,9.0,11.0,NaN,NaN
4864,2013-01-10,1,1,9,2013,1,2,10,3,0,12.0,14.0,NaN,NaN


In [47]:
# Rolling Features
windows = [7, 14, 28]

for window in windows:
    df[f"rolling_mean_{window}"] = (
        df.groupby(['store', 'item'])['sales']
          .shift(1)
          .rolling(window)
          .mean()
    )

In [48]:
df.head(10)

,date,store,item,sales,year,month,week,day,dayofweek,is_weekend,sales_lag_1,sales_lag_7,sales_lag_14,sales_lag_28,rolling_mean_7,rolling_mean_14,rolling_mean_28
0,2013-01-01,1,1,13,2013,1,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
699,2013-01-02,1,1,11,2013,1,1,2,2,0,13.0,NaN,NaN,NaN,NaN,NaN,NaN
1062,2013-01-03,1,1,14,2013,1,1,3,3,0,11.0,NaN,NaN,NaN,NaN,NaN,NaN
1861,2013-01-04,1,1,13,2013,1,1,4,4,0,14.0,NaN,NaN,NaN,NaN,NaN,NaN
2291,2013-01-05,1,1,10,2013,1,1,5,5,1,13.0,NaN,NaN,NaN,NaN,NaN,NaN
2659,2013-01-06,1,1,12,2013,1,1,6,6,1,10.0,NaN,NaN,NaN,NaN,NaN,NaN
3238,2013-01-07,1,1,10,2013,1,2,7,0,0,12.0,NaN,NaN,NaN,NaN,NaN,NaN
3587,2013-01-08,1,1,9,2013,1,2,8,1,0,10.0,13.0,NaN,NaN,11.857143,NaN,NaN
4256,2013-01-09,1,1,12,2013,1,2,9,2,0,9.0,11.0,NaN,NaN,11.285714,NaN,NaN
4864,2013-01-10,1,1,9,2013,1,2,10,3,0,12.0,14.0,NaN,NaN,11.428571,NaN,NaN


### 3. Creating Aggregate Features

In [49]:
# Store-Level Aggregate Feature
df['store_avg_sales'] = (
    df.groupby('store')['sales']
      .transform('mean')
)

In [50]:
df

,date,store,item,sales,year,month,week,day,dayofweek,is_weekend,sales_lag_1,sales_lag_7,sales_lag_14,sales_lag_28,rolling_mean_7,rolling_mean_14,rolling_mean_28,store_avg_sales
0,2013-01-01,1,1,13,2013,1,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.268379
699,2013-01-02,1,1,11,2013,1,1,2,2,0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,47.268379
1062,2013-01-03,1,1,14,2013,1,1,3,3,0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,47.268379
1861,2013-01-04,1,1,13,2013,1,1,4,4,0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,47.268379
2291,2013-01-05,1,1,10,2013,1,1,5,5,1,13.0,NaN,NaN,NaN,NaN,NaN,NaN,47.268379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910872,2017-12-27,10,50,63,2017,12,52,27,2,0,41.0,51.0,67.0,76.0,61.000000,63.071429,64.142857,58.709288
911260,2017-12-28,10,50,59,2017,12,52,28,3,0,63.0,63.0,72.0,73.0,62.714286,62.785714,63.678571,58.709288
911707,2017-12-29,10,50,74,2017,12,52,29,4,0,59.0,75.0,72.0,66.0,62.142857,61.857143,63.178571,58.709288
912007,2017-12-30,10,50,62,2017,12,52,30,5,1,74.0,70.0,52.0,49.0,62.000000,62.000000,63.464286,58.709288


In [51]:
# Item-Level Aggregate Feature
df['item_avg_sales'] = (
    df.groupby('item')['sales']
      .transform('mean')
)

In [52]:
df

,date,store,item,sales,year,month,week,day,dayofweek,is_weekend,sales_lag_1,sales_lag_7,sales_lag_14,sales_lag_28,rolling_mean_7,rolling_mean_14,rolling_mean_28,store_avg_sales,item_avg_sales
0,2013-01-01,1,1,13,2013,1,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.268379,21.981599
699,2013-01-02,1,1,11,2013,1,1,2,2,0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,47.268379,21.981599
1062,2013-01-03,1,1,14,2013,1,1,3,3,0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,47.268379,21.981599
1861,2013-01-04,1,1,13,2013,1,1,4,4,0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,47.268379,21.981599
2291,2013-01-05,1,1,10,2013,1,1,5,5,1,13.0,NaN,NaN,NaN,NaN,NaN,NaN,47.268379,21.981599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910872,2017-12-27,10,50,63,2017,12,52,27,2,0,41.0,51.0,67.0,76.0,61.000000,63.071429,64.142857,58.709288,65.882202
911260,2017-12-28,10,50,59,2017,12,52,28,3,0,63.0,63.0,72.0,73.0,62.714286,62.785714,63.678571,58.709288,65.882202
911707,2017-12-29,10,50,74,2017,12,52,29,4,0,59.0,75.0,72.0,66.0,62.142857,61.857143,63.178571,58.709288,65.882202
912007,2017-12-30,10,50,62,2017,12,52,30,5,1,74.0,70.0,52.0,49.0,62.000000,62.000000,63.464286,58.709288,65.882202


In [53]:
df = df.dropna()

In [54]:
df

,date,store,item,sales,year,month,week,day,dayofweek,is_weekend,sales_lag_1,sales_lag_7,sales_lag_14,sales_lag_28,rolling_mean_7,rolling_mean_14,rolling_mean_28,store_avg_sales,item_avg_sales
14425,2013-01-29,1,1,6,2013,1,5,29,1,0,11.0,7.0,5.0,13.0,10.428571,10.642857,10.714286,47.268379,21.981599
14518,2013-01-30,1,1,9,2013,1,5,30,2,0,6.0,9.0,7.0,11.0,10.285714,10.714286,10.464286,47.268379,21.981599
15210,2013-01-31,1,1,13,2013,1,5,31,3,0,9.0,8.0,16.0,14.0,10.285714,10.857143,10.392857,47.268379,21.981599
15626,2013-02-01,1,1,11,2013,2,5,1,4,0,13.0,14.0,7.0,13.0,11.000000,10.642857,10.357143,47.268379,21.981599
16029,2013-02-02,1,1,21,2013,2,5,2,5,1,11.0,12.0,18.0,10.0,10.571429,10.928571,10.285714,47.268379,21.981599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910872,2017-12-27,10,50,63,2017,12,52,27,2,0,41.0,51.0,67.0,76.0,61.000000,63.071429,64.142857,58.709288,65.882202
911260,2017-12-28,10,50,59,2017,12,52,28,3,0,63.0,63.0,72.0,73.0,62.714286,62.785714,63.678571,58.709288,65.882202
911707,2017-12-29,10,50,74,2017,12,52,29,4,0,59.0,75.0,72.0,66.0,62.142857,61.857143,63.178571,58.709288,65.882202
912007,2017-12-30,10,50,62,2017,12,52,30,5,1,74.0,70.0,52.0,49.0,62.000000,62.000000,63.464286,58.709288,65.882202


### Feature Engineering Done

In [55]:
# Saving Dataset
df.to_csv("../data/processed/featured_data.csv", index=False)